In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
params_grid= [
    {'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
    {'boostrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}
]

In [7]:
forest_reg=RandomForestRegressor()
grid_search=GridSearchCV(forest_reg,param_grid=params_grid,cv=5,scoring='neg_mean_squared_error')

In [8]:
grid_search.fit()

TypeError: BaseSearchCV.fit() missing 1 required positional argument: 'X'

In [29]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,mean_squared_error



In [30]:
data = {
    'date': ['2022-01-01', '2022-01-15', '2022-02-05', '2022-02-20'],
    'value': [1, 2, 3, 4],
    'label': ['A', 'B', 'A', 'B']
}

df = pd.DataFrame(data)

In [31]:
class DateConverter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed['date'] = pd.to_datetime(X_transformed['date'])
        return X_transformed
    
    
class DateFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed['day_of_week'] = X_transformed['date'].dt.dayofweek
        X_transformed['month'] = X_transformed['date'].dt.month
        return X_transformed.drop('date', axis=1)

In [32]:
date_features_pipeline = Pipeline([
    ('date_converter', DateConverter()),
    ('date_features', DateFeaturesExtractor()),
    ('scaler', StandardScaler())
])


In [33]:

preprocessor = ColumnTransformer(
    transformers=[
        ('date', date_features_pipeline, ['date']),
        ('other', OneHotEncoder(), ['label'])
    ],
    remainder='passthrough'
)

In [34]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])
full_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('date',
                                                  Pipeline(steps=[('date_converter',
                                                                   DateConverter()),
                                                                  ('date_features',
                                                                   DateFeaturesExtractor()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['date']),
                                                 ('other', OneHotEncoder(),
                                                  ['label'])])),
                ('classifier', RandomForestClassifier())])

In [74]:
X = df[['date', 'label']]
y = df['value']

In [75]:
y

0    1
1    2
2    3
3    4
Name: value, dtype: int64

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
y_test

1    2
Name: value, dtype: int64

In [78]:
y_train

3    4
0    1
2    3
Name: value, dtype: int64

In [79]:
X_train.head()

,date,label
3,2022-02-20,B
0,2022-01-01,A
2,2022-02-05,A


In [80]:
X_test

,date,label
1,2022-01-15,B


In [81]:
# Fit and predict using the full pipeline
full_pipeline.fit(X_train, y_train)
y_pred = full_pipeline.predict(X_test)

In [82]:
# Evaluate the model
accuracy = mean_squared_error(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 1.00


In [83]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

In [84]:
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

In [85]:
voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard'
)


In [86]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('voting_clf',voting_clf)
])
full_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('date',
                                                  Pipeline(steps=[('date_converter',
                                                                   DateConverter()),
                                                                  ('date_features',
                                                                   DateFeaturesExtractor()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['date']),
                                                 ('other', OneHotEncoder(),
                                                  ['label'])])),
                ('voting_clf',
                 VotingClassifier(estimators=[('lr', LogisticRegression()),
                                              ('rf', RandomForestClassifier()),
                                              ('svc', SVC())]))])

In [87]:
y_pred=full_pipeline.fit(X_train,y_train)

In [88]:
y_pred.predict(X_test)

array([1], dtype=int64)

In [89]:
# Fit and predict using the full pipeline
full_pipeline.fit(X_train, y_train)
y_pred = full_pipeline.predict(X_test)

In [90]:
accuracy = mean_squared_error(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 1.00


In [91]:
from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(X_test,y_pred))

ValueError: could not convert string to float: '2022-02-20'